In [ ]:
from sklearn.svm import SVC,LinearSVC
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

In [ ]:
IMG_SIZE=224
test_transforms = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)

In [ ]:
use_cuda = torch.cuda.is_available()
device = 'cuda' if use_cuda else 'cpu'

In [ ]:
PATH="/kaggle/input/efficientnet-weights-on-vgg/EfficientNetb0.pth"

In [ ]:
feature_extractor_model = torch.load(PATH)
feature_extractor_model.classifier=torch.nn.Identity()
feature_extractor_model.eval()
feature_extractor_model=feature_extractor_model.to(device)
print(feature_extractor_model)

In [ ]:
DATA_DIR="/kaggle/input/ravdess-frames/RAVDESS"
emotion_to_index = {'Neutral':1, 'Happiness':2, 'Sadness':3, 'Suprise':4, 'Fear':5, 'Disgust':6, 'Anger':7}

In [ ]:
import os

In [ ]:
def get_features(data_dir):
    filename2features={}
    for filename in os.listdir(data_dir):
        frames_file=os.path.join(data_dir,filename)
        for frames_dir in os.listdir(frames_file):
            X_global_features,X_isface=[],[]
            imgs=[]
            frames_full_dir = os.path.join(frames_file, frames_dir)
            for img_name in os.listdir(frames_full_dir):
                img = Image.open(os.path.join(frames_full_dir,img_name))
                img_tensor = test_transforms(img)
                imgs.append(img_tensor)
                X_isface.append(filename)
                if(len(imgs)==20):
                    scores = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
                    scores=scores.data.cpu().numpy()
                    X_global_features=scores


            X_isface=np.array(X_isface)
            filename2features[frames_dir]=(X_global_features,X_isface)
    return filename2features

In [ ]:
filename2features=get_features(DATA_DIR)

In [ ]:
len(filename2features['05_371'][0][0])

In [ ]:
def create_dataset(filename2features,data_dir):
    x = []
    y = []
    for filename in os.listdir(data_dir):
        print(filename)
        frames_file=os.path.join(data_dir,filename)
        for frames_dir in os.listdir(frames_file):
            features=filename2features[frames_dir]
            total_features=None
            cur_features=features[0]
            mean_features = (np.mean(cur_features, axis=0))
            std_features = (np.std(cur_features, axis=0))
            max_features = (np.max(cur_features, axis=0))
            min_features = (np.min(cur_features, axis=0))

            # join several features together
            feature = np.concatenate((mean_features, std_features, min_features, max_features), axis=None)                   
            total_features=feature
            
            if total_features is not None:
                x.append(total_features)
                y.append(filename)
    x=np.array(x)
    y=np.array(y)
    print(x.shape,y.shape)
    return x,y


In [ ]:
x,y=create_dataset(filename2features,DATA_DIR)

In [ ]:
x[0]

In [ ]:
from sklearn import preprocessing
x_norm=preprocessing.normalize(x,norm='l2')

In [ ]:
x_norm[0]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_norm, y, test_size=0.25, random_state=42)

In [ ]:
print(len(x_train))
print(len(x_test))

In [ ]:
from sklearn import svm,metrics,preprocessing
clf = svm.LinearSVC(C=4.1)   
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(f"Complete accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
!pip install joblib

In [ ]:
import joblib
joblib.dump(clf, 'efficientnet_video_model.pkl')